# 1. Create the Dataframe of Toronto postal code

In [1]:
import pandas as pd

In [2]:
import requests
from bs4 import BeautifulSoup
res=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup=BeautifulSoup(res.text,'html.parser')
tables = soup.select('table')
df_list = []
for table in tables:
    df_list.append(pd.concat(pd.read_html(table.prettify())))
df1 = pd.concat(df_list)

df=df1[['Postal Code','Borough','Neighbourhood']]
df=df.dropna(subset=['Postal Code'])
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
df=df[df.Borough!='Not assigned']
df=df.reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


I have created a pandas dataframe of Toronto postal codes from wikipedia and droped the unassigned Borough.

In [4]:
df.shape

(103, 3)

# 2. Add the Geographical Coordinates

In [5]:
geo=pd.read_csv('http://cocl.us/Geospatial_data')
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
df=pd.merge(left=df,right=geo)
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# 3. Explore and cluster the Neighbourhoods in Tonronto

In [7]:
!conda install -c conda-forge folium=0.5.0 --yes
import numpy as np
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import webbrowser

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

#### create a map of toronto

In [77]:
address='Toronto'
geolocator=Nominatim(user_agent="np_explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
latitude,longitude

(43.6534817, -79.3839347)

In [76]:
map_toronto=folium.Map(location=[latitude,longitude],zoom_start=10)


In [11]:
for lat,lon,borough,neighborhood in zip(df['Latitude'],df['Longitude'],df['Borough'],df['Neighbourhood']):
    label='{},{}'.format(neighborhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,lon],
                       radius=5,
                       popup=label,
                       color='blue',
                       fill=True,
                       fill_color='#3186cc',
                       fill_opacity=0.7,
                       parse_html=False).add_to(map_toronto)

Define Foursquare Credentials and Version

In [12]:
CLIENT_ID='ZI4AGYQV250W0XUXT0IK015MFBHSRU2WP5DSJHTG1VYCGFRL'

CLIENT_SECRET='JBLAAX3KT3N1VWE0I41ZZGW5UXK3R2GKN5KQGSPXIHMK3WIF'

Version=20180605


Explore the first Neighbourhood in our Dataframe

In [13]:
neigh_name=df.loc[0,'Neighbourhood']
neigh_lat=df.loc[0,'Latitude']
neigh_lon=df.loc[0,'Longitude']
print('Latitude and Longitude value of {} is {},{}.'.format(neigh_name,neigh_lat,neigh_lon))

Latitude and Longitude value of Parkwoods is 43.7532586,-79.3296565.


In [14]:
LIMIT=100
radius=500
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,Version,neigh_lat,neigh_lon,radius,LIMIT)

In [15]:
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f338c66022b1022e2ebc9f1'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

clean the json and structure it into a pandas dataframe

In [16]:
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']
  

In [17]:
venues=results['response']['groups'][0]['items']
nearby_venues=json_normalize(venues)
filtered_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues=nearby_venues.loc[:,filtered_columns]
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type,axis=1)
nearby_venues.columns=[col.split(".")[-1] for col in nearby_venues.columns]

In [24]:
def getNearbyVenues(names,latitudes,longitudes,radius=500):
    venues_list=[]
    for name, lat, lon in zip(names,latitudes,longitudes):
        print(name)
        
        url='http://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,
                                                                                                                                Version,
                                                                                                                                lat,
                                                                                                                                lon,
                                                                                                                                radius,
                                                                                                                                LIMIT)
        results=requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(name,lat,lon,v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns=['Neighborhood','Neighborhood Latitude','Neighborhood Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category']
    
    return(nearby_venues)

toronto_venues=getNearbyVenues(names=df['Neighbourhood'],latitudes=df['Latitude'],longitudes=df['Longitude'])

print(toronto_venues.shape)
toronto_venues.head()

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

# 4. Analyze Each Neighbourhood

In [28]:
toronto_onehot=pd.get_dummies(toronto_venues[['Venue Category']],prefix="",prefix_sep="")
toronto_onehot['Neighbourhood']=toronto_venues['Neighborhood']
fixed_columns=[toronto_onehot.columns[-1]]+list(toronto_onehot.columns[:-1])
toronto_onehot=toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
toronto_grouped=toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
1,"Alderwood, Long Branch",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
3,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
4,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
5,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
6,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
7,"Brockton, Parkdale Village, Exhibition Place",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.038462
8,"Business reply mail Processing Centre, South C...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.058824,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000


In [36]:
num_top_venues=5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp=toronto_grouped[toronto_grouped['Neighbourhood']==hood].T.reset_index()
    temp.columns=['venue','freq']
    temp=temp.iloc[1:]
    temp['freq']=temp['freq'].astype(float)
    temp=temp.round({'freq':2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venues))

----Agincourt----
                       venue  freq
0               Skating Rink  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3  Latin American Restaurant  0.25
4         Mexican Restaurant  0.00
----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.33
1             Pub  0.17
2  Sandwich Place  0.17
3             Gym  0.17
4     Coffee Shop  0.17
----Bathurst Manor, Wilson Heights, Downsview North----
                venue  freq
0         Coffee Shop  0.09
1                Bank  0.09
2       Deli / Bodega  0.04
3  Frozen Yogurt Shop  0.04
4    Sushi Restaurant  0.04
----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4    Accessories Store  0.00
----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.09
1      Sandwich Place  0.09
2          Restaurant  0.09
3         Coffee Shop

                venue  freq
0               Field  0.25
1        Tennis Court  0.25
2        Hockey Arena  0.25
3               Trail  0.25
4  Mexican Restaurant  0.00
----India Bazaar, The Beaches West----
                  venue  freq
0           Pizza Place  0.10
1        Sandwich Place  0.10
2                  Park  0.05
3                   Pub  0.05
4  Fast Food Restaurant  0.05
----Kennedy Park, Ionview, East Birchmount Park----
                venue  freq
0    Department Store   0.2
1          Hobby Shop   0.2
2         Bus Station   0.2
3         Coffee Shop   0.2
4  Chinese Restaurant   0.2
----Kensington Market, Chinatown, Grange Park----
                           venue  freq
0                    Coffee Shop  0.06
1                           Café  0.06
2          Vietnamese Restaurant  0.05
3  Vegetarian / Vegan Restaurant  0.05
4             Mexican Restaurant  0.05
----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
                 venue  freq


                venue  freq
0         Coffee Shop  0.12
1                 Pub  0.12
2        Liquor Store  0.06
3  Light Rail Station  0.06
4          Sports Bar  0.06
----The Annex, North Midtown, Yorkville----
            venue  freq
0            Café  0.15
1  Sandwich Place  0.15
2     Coffee Shop  0.10
3        Pharmacy  0.05
4      Donut Shop  0.05
----The Beaches----
               venue  freq
0                Pub  0.25
1  Health Food Store  0.25
2       Neighborhood  0.25
3              Trail  0.25
4  Accessories Store  0.00
----The Danforth West, Riverdale----
                    venue  freq
0        Greek Restaurant  0.17
1             Coffee Shop  0.10
2      Italian Restaurant  0.07
3          Ice Cream Shop  0.05
4  Furniture / Home Store  0.05
----The Kingsway, Montgomery Road, Old Mill North----
                       venue  freq
0                      River  0.33
1                       Park  0.33
2                 Smoke Shop  0.33
3          Accessories Store  0.00
4  M

In [37]:
def return_most_common_venues(row,num_top_venues):
    row_categories=row.iloc[1:]
    row_categories_sorted=row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [57]:
num_top_venues=10
indicators=['st','nd','rd']
columns=['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{}Most Common Venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
neighbourhood_venue_sorted=pd.DataFrame(columns=columns)
neighbourhood_venue_sorted['Neighbourhood']=toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhood_venue_sorted.iloc[ind,1:]=return_most_common_venues(toronto_grouped.iloc[ind,:],num_top_venues)

neighbourhood_venue_sorted.head()

,Neighbourhood,1stMost Common Venue,2ndMost Common Venue,3rdMost Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
1,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Sandwich Place,Pub,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Ice Cream Shop,Pizza Place,Restaurant,Deli / Bodega,Supermarket,Fried Chicken Joint,Frozen Yogurt Shop,Middle Eastern Restaurant
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
4,"Bedford Park, Lawrence Manor East",Restaurant,Coffee Shop,Italian Restaurant,Sandwich Place,Thai Restaurant,Pizza Place,Butcher,Café,Pub,Sushi Restaurant


# 5.Cluster Neighbourhoods

In [58]:
kclusters=5
toronto_grouped_clustering=toronto_grouped.drop('Neighbourhood',1)
kmeans=KMeans(n_clusters=kclusters,random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10]

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [59]:
neighbourhood_venue_sorted.insert(0,'Cluster Labels',kmeans.labels_)
toronto_merged=df

In [67]:
toronto_merged=pd.concat([toronto_merged,neighbourhood_venue_sorted])

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [68]:
toronto_merged.head()

,10th Most Common Venue,1stMost Common Venue,2ndMost Common Venue,3rdMost Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,Borough,Cluster Labels,Latitude,Longitude,Neighbourhood,Postal Code
0,Doner Restaurant,Food & Drink Shop,Park,Construction & Landscaping,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,North York,1.0,43.753259,-79.329656,Parkwoods,M3A
1,Discount Store,Hockey Arena,French Restaurant,Pizza Place,Coffee Shop,Portuguese Restaurant,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,North York,1.0,43.725882,-79.315572,Victoria Village,M4A
2,Beer Store,Coffee Shop,Bakery,Café,Pub,Park,Breakfast Spot,Restaurant,Theater,Spa,Downtown Toronto,1.0,43.654260,-79.360636,"Regent Park, Harbourfront",M5A
3,Drugstore,Clothing Store,Accessories Store,Vietnamese Restaurant,Event Space,Furniture / Home Store,Coffee Shop,Boutique,Eastern European Restaurant,Dumpling Restaurant,North York,1.0,43.718518,-79.464763,"Lawrence Manor, Lawrence Heights",M6A
4,Park,Coffee Shop,Sushi Restaurant,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Café,Portuguese Restaurant,Persian Restaurant,Downtown Toronto,1.0,43.662301,-79.389494,"Queen's Park, Ontario Provincial Government",M7A


In [80]:
import webbrowser
map_clusters=folium.Map(location=[latitude,longitude],zoom_start=11)
map_clusters


In [83]:
x=np.arange(kclusters)
ys=[i+x+(i*x)**2 for i in range(kclusters)]
color_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow=[colors.rgb2hex(i) for i in color_array]

Cluster 1

In [86]:
toronto_merged.loc[toronto_merged['Cluster Labels']==0,toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,1stMost Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,Borough,Cluster Labels,Latitude,Longitude,Neighbourhood,Postal Code
50,Pizza Place,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,North York,0.0,43.756303,-79.565963,Humber Summit,M9L
70,Pizza Place,Sandwich Place,Middle Eastern Restaurant,Discount Store,Yoga Studio,Dessert Shop,Etobicoke,0.0,43.696319,-79.532242,Westmount,M9P
77,Pizza Place,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Etobicoke,0.0,43.688905,-79.554724,"Kingsview Village, St. Phillips, Martin Grove ...",M9R
93,Pizza Place,Pub,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Etobicoke,0.0,43.602414,-79.543484,"Alderwood, Long Branch",M8W
1,Pizza Place,Pub,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,NaN,0.0,NaN,NaN,"Alderwood, Long Branch",NaN
38,Pizza Place,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,NaN,0.0,NaN,NaN,Humber Summit,NaN
44,Pizza Place,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,NaN,0.0,NaN,NaN,"Kingsview Village, St. Phillips, Martin Grove ...",NaN
86,Pizza Place,Sandwich Place,Middle Eastern Restaurant,Discount Store,Yoga Studio,Dessert Shop,NaN,0.0,NaN,NaN,Westmount,NaN
1,Pizza Place,Pub,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,NaN,0.0,NaN,NaN,"Alderwood, Long Branch",NaN
38,Pizza Place,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,NaN,0.0,NaN,NaN,Humber Summit,NaN


cluster 2

In [87]:
toronto_merged.loc[toronto_merged['Cluster Labels']==1,toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,1stMost Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,Borough,Cluster Labels,Latitude,Longitude,Neighbourhood,Postal Code
0,Food & Drink Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,North York,1.0,43.753259,-79.329656,Parkwoods,M3A
1,Hockey Arena,Portuguese Restaurant,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,North York,1.0,43.725882,-79.315572,Victoria Village,M4A
2,Coffee Shop,Park,Breakfast Spot,Restaurant,Theater,Spa,Downtown Toronto,1.0,43.654260,-79.360636,"Regent Park, Harbourfront",M5A
3,Clothing Store,Furniture / Home Store,Coffee Shop,Boutique,Eastern European Restaurant,Dumpling Restaurant,North York,1.0,43.718518,-79.464763,"Lawrence Manor, Lawrence Heights",M6A
4,Coffee Shop,Smoothie Shop,Sandwich Place,Café,Portuguese Restaurant,Persian Restaurant,Downtown Toronto,1.0,43.662301,-79.389494,"Queen's Park, Ontario Provincial Government",M7A
7,Gym,Restaurant,Coffee Shop,Shopping Mall,Dim Sum Restaurant,Italian Restaurant,North York,1.0,43.745906,-79.352188,Don Mills,M3B
8,Pizza Place,Intersection,Pharmacy,Café,Breakfast Spot,Bank,East York,1.0,43.706397,-79.309937,"Parkview Hill, Woodbine Gardens",M4B
9,Clothing Store,Cosmetics Shop,Bubble Tea Shop,Theater,Diner,Electronics Store,Downtown Toronto,1.0,43.657162,-79.378937,"Garden District, Ryerson",M5B
10,Metro Station,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,North York,1.0,43.709577,-79.445073,Glencairn,M6B
12,Bar,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Scarborough,1.0,43.784535,-79.160497,"Rouge Hill, Port Union, Highland Creek",M1C


cluster 3

In [88]:
toronto_merged.loc[toronto_merged['Cluster Labels']==2,toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,1stMost Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,Borough,Cluster Labels,Latitude,Longitude,Neighbourhood,Postal Code
21,Park,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,York,2.0,43.689026,-79.453512,Caledonia-Fairbanks,M6E
35,Park,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,East York,2.0,43.685347,-79.338106,"East Toronto, Broadview North (Old East York)",M4J
52,Park,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,North York,2.0,43.789053,-79.408493,"Willowdale, Newtonbrook",M2M
64,Park,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,York,2.0,43.706876,-79.518188,Weston,M9N
66,Park,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,North York,2.0,43.752758,-79.400049,York Mills West,M2P
85,Park,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Scarborough,2.0,43.815252,-79.284577,"Milliken, Agincourt North, Steeles East, L'Amo...",M1V
91,Park,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Downtown Toronto,2.0,43.679563,-79.377529,Rosedale,M4W
10,Park,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,NaN,2.0,NaN,NaN,Caledonia-Fairbanks,NaN
26,Park,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,NaN,2.0,NaN,NaN,"East Toronto, Broadview North (Old East York)",NaN
50,Park,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,NaN,2.0,NaN,NaN,"Milliken, Agincourt North, Steeles East, L'Amo...",NaN


cluster 4

In [90]:
toronto_merged.loc[toronto_merged['Cluster Labels']==3,toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,1stMost Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,Borough,Cluster Labels,Latitude,Longitude,Neighbourhood,Postal Code
6,Fast Food Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Scarborough,3.0,43.806686,-79.194353,"Malvern, Rouge",M1B
49,Fast Food Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,NaN,3.0,NaN,NaN,"Malvern, Rouge",NaN
49,Fast Food Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,NaN,3.0,NaN,NaN,"Malvern, Rouge",NaN


cluster 5

In [91]:
toronto_merged.loc[toronto_merged['Cluster Labels']==4,toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,1stMost Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,Borough,Cluster Labels,Latitude,Longitude,Neighbourhood,Postal Code
57,Baseball Field,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,North York,4.0,43.724766,-79.532242,"Humberlea, Emery",M9M
62,Garden,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Central Toronto,4.0,43.711695,-79.416936,Roselawn,M5N
101,Home Service,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Etobicoke,4.0,43.636258,-79.498509,"Old Mill South, King's Mill Park, Sunnylea, Hu...",M8Y
39,Baseball Field,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,NaN,4.0,NaN,NaN,"Humberlea, Emery",NaN
58,Home Service,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,NaN,4.0,NaN,NaN,"Old Mill South, King's Mill Park, Sunnylea, Hu...",NaN
66,Garden,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,NaN,4.0,NaN,NaN,Roselawn,NaN
39,Baseball Field,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,NaN,4.0,NaN,NaN,"Humberlea, Emery",NaN
58,Home Service,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,NaN,4.0,NaN,NaN,"Old Mill South, King's Mill Park, Sunnylea, Hu...",NaN
66,Garden,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,NaN,4.0,NaN,NaN,Roselawn,NaN
